<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest

# Helper to mimic TopLevelFixture - ensures settings are restored
class QuantLibTestCase(unittest.TestCase):
    def setUp(self):
        self.saved_settings = ql.SavedSettings()
        # Set a default evaluation date if tests rely on it
        # For these index tests, using a fixed date helps ensure reproducibility.
        self.evaluation_date = ql.Date(15, ql.May, 2020) # Example fixed date
        ql.Settings.instance().evaluationDate = self.evaluation_date
        # Clear fixings before each test to ensure a clean state,
        # similar to how C++ tests might operate in a fresh environment
        # or use TopLevelFixture to reset IndexManager.
        # Note: Individual tests might add fixings.
        ql.IndexManager.instance().clearHistories()


    def tearDown(self):
        # Clear fixings after each test as well, so one test doesn't affect another
        ql.IndexManager.instance().clearHistories()
        self.saved_settings = None # Restores settings

class IndexTests(QuantLibTestCase):

    def testFixingObservability(self):
        self.subTestName = "Testing observability of index fixings..."
        # print(self.subTestName) # Optional: for verbose output

        i1 = ql.Euribor6M()
        i2 = ql.BMAIndex()

        # Python's equivalent of Flag can be a simple class or a lambda
        class FlagObserver(ql.Observer):
            def __init__(self):
                super(FlagObserver, self).__init__()
                self.flag = False
            def update(self):
                self.flag = True
            def isUp(self):
                return self.flag
            def lower(self):
                self.flag = False

        f1 = FlagObserver()
        f1.registerWith(i1) # Observable.registerObserver
        f1.lower()

        f2 = FlagObserver()
        f2.registerWith(i2)
        f2.lower()

        today = ql.Settings.instance().evaluationDate

        # Use the same index instances i1 and i2 for adding fixings
        # as those f1 and f2 are registered with.
        # The C++ test creates new instances euribor and bma, but then
        # tests f1 and f2 which were registered with i1 and i2.
        # This works in C++ because addFixing on an index instance updates
        # the global IndexManager, which then notifies all observers of
        # *any* instance of that specific index (e.g., any Euribor6M instance).

        d1 = today
        while not i1.isValidFixingDate(d1):
            d1 = d1 + ql.Period(1, ql.Days) # Increment, C++ was d1++

        i1.addFixing(d1, -0.003)
        self.assertTrue(f1.isUp(), "Observer was not notified of added Euribor fixing")

        d2 = today
        while not i2.isValidFixingDate(d2):
            d2 = d2 + ql.Period(1, ql.Days)

        i2.addFixing(d2, 0.01)
        self.assertTrue(f2.isUp(), "Observer was not notified of added BMA fixing")

    def testFixingHasHistoricalFixing(self):
        self.subTestName = "Testing if index has historical fixings..."
        # print(self.subTestName)

        def check_fixing(index_name_test, expected_result, actual_result):
            self.assertEqual(expected_result, actual_result,
                             f"Historical fixing test failed for {index_name_test}. "
                             f"Expected: {expected_result}, Got: {actual_result}")

        fixing_found = True
        fixing_not_found = False

        euribor3M = ql.Euribor3M()
        euribor6M_main = ql.Euribor6M() # The one we add fixing to
        euribor6M_alias = ql.Euribor6M() # Another instance of the same index

        today = ql.Settings.instance().evaluationDate
        # Find a valid fixing date for Euribor6M (could be in the past)
        fixing_date_6M = today
        while not euribor6M_main.isValidFixingDate(fixing_date_6M):
            fixing_date_6M = fixing_date_6M - ql.Period(1, ql.Days) # Go backwards

        euribor6M_main.addFixing(fixing_date_6M, 0.01)

        # Test Euribor3M (no fixing added for this date)
        name_e3m = euribor3M.name()
        check_fixing(name_e3m, fixing_not_found, euribor3M.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e3m + " (Manager)", fixing_not_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e3m, fixing_date_6M))
        check_fixing(name_e3m.upper() + " (Manager)", fixing_not_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e3m.upper(), fixing_date_6M))
        check_fixing(name_e3m.lower() + " (Manager)", fixing_not_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e3m.lower(), fixing_date_6M))

        # Test Euribor6M (fixing was added)
        name_e6m = euribor6M_main.name()
        check_fixing(name_e6m, fixing_found, euribor6M_main.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e6m + " (alias)", fixing_found, euribor6M_alias.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e6m + " (Manager)", fixing_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e6m, fixing_date_6M))
        check_fixing(name_e6m.upper() + " (Manager)", fixing_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e6m.upper(), fixing_date_6M))
        check_fixing(name_e6m.lower() + " (Manager)", fixing_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e6m.lower(), fixing_date_6M))

        # Clear histories and re-test
        ql.IndexManager.instance().clearHistories()

        check_fixing(name_e3m + " (cleared)", fixing_not_found, euribor3M.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e3m + " (Manager, cleared)", fixing_not_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e3m, fixing_date_6M))
        # ... (upper/lower case for e3m after clear - should also be not_found)

        check_fixing(name_e6m + " (cleared)", fixing_not_found, euribor6M_main.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e6m + " (alias, cleared)", fixing_not_found, euribor6M_alias.hasHistoricalFixing(fixing_date_6M))
        check_fixing(name_e6m + " (Manager, cleared)", fixing_not_found,
                     ql.IndexManager.instance().hasHistoricalFixing(name_e6m, fixing_date_6M))
        # ... (upper/lower case for e6m after clear - should also be not_found)


    def testTenorNormalization(self):
        self.subTestName = "Testing that interest-rate index tenor is normalized correctly..."
        # print(self.subTestName)

        # IborIndex(familyName, tenor, settlementDays, currency, fixingCalendar,
        #           convention, endOfMonth, dayCounter, fixingCurve=Handle())
        # Use default empty Handle for fixingCurve if not needed for name/maturityDate tests.
        dummy_curve = ql.YieldTermStructureHandle() # Or a flat curve

        i12m = ql.IborIndex("foo", ql.Period(12, ql.Months), 2, ql.EURCurrency(),
                            ql.TARGET(), ql.Following, False, ql.Actual360(), dummy_curve)
        i1y = ql.IborIndex("foo", ql.Period(1, ql.Years), 2, ql.EURCurrency(),
                           ql.TARGET(), ql.Following, False, ql.Actual360(), dummy_curve)

        # Name check relies on how QL constructs the name string from tenor.
        # If Period(12, Months) and Period(1, Years) normalize to the same string part, they'll match.
        # QL's Index::name() often includes a normalized tenor string.
        self.assertEqual(i12m.name(), i1y.name(),
                         "12M index and 1Y index yield different names")

        # Test maturity date for short tenors
        i6d = ql.IborIndex("foo", ql.Period(6, ql.Days), 2, ql.EURCurrency(),
                           ql.TARGET(), ql.Following, False, ql.Actual360(), dummy_curve)
        i7d = ql.IborIndex("foo", ql.Period(7, ql.Days), 2, ql.EURCurrency(),
                           ql.TARGET(), ql.Following, False, ql.Actual360(), dummy_curve)

        test_date = ql.Date(28, ql.April, 2023)
        maturity6d = i6d.maturityDate(test_date)
        maturity7d = i7d.maturityDate(test_date)

        self.assertLess(maturity6d, maturity7d,
                        f"Inconsistent maturity dates and tenors: "
                        f"Maturity 6D: {maturity6d}, Maturity 7D: {maturity7d}")


if __name__ == '__main__':
    print("Running QuantLib-Python IndexTests...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)